In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Ceará - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_CE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Ceará - Consumo de Cimento (t)
0,2003-1,8.274459,0.301376,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,...,21.474686,72.364001,3.070554,0.684040,6.965352e+07,4.525724e+06,7.532751,6.293821e+07,107.673000,81.265
1,2003-2,8.268512,0.301393,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,...,21.474876,72.363165,3.069036,0.684476,6.974354e+07,4.530359e+06,7.535808,6.297178e+07,107.237962,65.200
2,2003-3,8.262564,0.301309,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,...,21.474987,72.362677,3.068150,0.684912,6.983356e+07,4.534994e+06,7.538865,6.300536e+07,108.742184,55.165
3,2003-4,8.256617,0.300917,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,...,21.474998,72.362625,3.068055,0.685347,6.992358e+07,4.539629e+06,7.541921,6.303893e+07,109.391112,53.928
4,2003-5,8.250670,0.300417,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,...,21.671734,73.524145,2.951262,0.685783,7.001361e+07,4.544265e+06,7.544978,6.307250e+07,111.602457,65.334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.535686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.741456,69.905378,2.592561,NaN,NaN,NaN,NaN,NaN,239.122997,186.918
236,2022-9,NaN,0.533990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.811662,69.012093,2.707176,NaN,NaN,NaN,NaN,NaN,239.220173,179.828
237,2022-10,NaN,0.530449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.859375,68.634594,2.725124,NaN,NaN,NaN,NaN,NaN,237.879214,174.940
238,2022-11,NaN,0.526253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20.927587,68.656183,2.666577,NaN,NaN,NaN,NaN,NaN,236.799325,182.597


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
0,-0.777612,-1.446563,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-0.308162,...,-0.416339,-0.010479,0.121023,0.812339,-2.089980,-1.710320,-1.784591,-2.264453,-2.078282,-1.711278
1,-0.782340,-1.445945,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.307589,...,-0.414806,-0.010207,0.120802,0.808988,-2.056200,-1.691964,-1.739638,-2.228847,-2.046399,-1.729252
2,-0.787067,-1.449072,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-0.307255,...,-0.413911,-0.010048,0.120673,0.807033,-2.022420,-1.673607,-1.694684,-2.193241,-2.014516,-1.667105
3,-0.791795,-1.463663,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-0.307219,...,-0.413815,-0.010031,0.120659,0.806823,-1.988641,-1.655251,-1.649730,-2.157635,-1.982633,-1.640295
4,-0.796522,-1.482248,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,0.148513,...,-0.548914,0.271648,0.428292,0.549090,-1.954861,-1.636895,-1.604776,-2.122029,-1.950750,-1.548935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.062332,1.412350,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,0.109414,...,-0.018091,-1.147183,-0.835529,0.121010,1.416090,1.084593,-1.281134,0.537648,0.738842,1.593689
188,1.063244,1.500856,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.401538,...,0.365944,-0.896043,-0.827279,0.123083,1.410646,1.073027,-1.282511,0.515683,0.720806,1.645498
189,1.064157,1.599064,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.615486,...,0.679715,-0.410313,-0.609153,-0.152425,1.405202,1.061462,-1.283888,0.493719,0.702770,1.698639
190,1.065069,1.702017,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,0.550554,...,1.435026,0.175592,-0.525307,-0.355465,1.399758,1.049897,-1.285265,0.471754,0.684734,1.730727


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      75.575
1      53.065
2      67.755
3      64.858
4      72.377
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Ceará - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
0,-0.777612,-1.446563,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-0.308162,...,-0.416339,-0.010479,0.121023,0.812339,-2.089980,-1.710320,-1.784591,-2.264453,-2.078282,-1.711278
1,-0.782340,-1.445945,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.307589,...,-0.414806,-0.010207,0.120802,0.808988,-2.056200,-1.691964,-1.739638,-2.228847,-2.046399,-1.729252
2,-0.787067,-1.449072,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-0.307255,...,-0.413911,-0.010048,0.120673,0.807033,-2.022420,-1.673607,-1.694684,-2.193241,-2.014516,-1.667105
3,-0.791795,-1.463663,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-0.307219,...,-0.413815,-0.010031,0.120659,0.806823,-1.988641,-1.655251,-1.649730,-2.157635,-1.982633,-1.640295
4,-0.796522,-1.482248,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,0.148513,...,-0.548914,0.271648,0.428292,0.549090,-1.954861,-1.636895,-1.604776,-2.122029,-1.950750,-1.548935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.423186,-0.021469,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,-0.010814,...,3.009262,1.041224,-0.576167,-1.275706,1.409954,1.190773,-0.838130,0.874129,1.017258,1.523635
158,1.399318,-0.031468,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,0.120633,...,2.843253,0.890818,-0.599064,-2.535161,1.414132,1.193287,-0.881990,0.871719,1.015919,1.541381
159,1.375451,-0.044253,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,-0.530269,...,2.364135,0.359401,-0.770590,-2.850455,1.418310,1.195800,-0.925849,0.869309,1.014581,1.523536
160,1.351584,-0.046806,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,-0.838306,...,2.115134,-0.381258,-1.211611,-2.417126,1.422488,1.198313,-0.969709,0.866899,1.013242,1.507028


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       75.575
1       53.065
2       67.755
3       64.858
4       72.377
        ...   
157    134.739
158    151.819
159    124.858
160    162.684
161    169.251
Name: Ceará - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
126,0.941464,1.223482,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,-0.480018,...,-0.309701,-0.286801,-0.041850,0.421468,0.365676,0.843485,0.634116,1.037548,1.010785,0.074200
127,0.969984,1.232594,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,-0.828855,...,0.124310,-0.600450,-0.514570,1.036869,0.371209,0.863067,0.604648,1.047639,1.024388,0.100668
128,0.998504,1.233295,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,-1.004415,...,0.483007,-0.631922,-0.721738,1.220308,0.376742,0.882650,0.575180,1.057730,1.037991,0.116036
129,1.027024,1.227338,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,-1.049285,...,0.761610,-0.465106,-0.736611,1.147966,0.382275,0.902233,0.545712,1.067821,1.051594,0.134124
130,1.055544,1.232230,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,-1.026862,...,0.789801,-0.019451,-0.411043,0.864373,0.387808,0.921816,0.516244,1.077913,1.065197,0.158476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.062332,1.412350,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,0.109414,...,-0.018091,-1.147183,-0.835529,0.121010,1.416090,1.084593,-1.281134,0.537648,0.738842,1.593689
188,1.063244,1.500856,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.401538,...,0.365944,-0.896043,-0.827279,0.123083,1.410646,1.073027,-1.282511,0.515683,0.720806,1.645498
189,1.064157,1.599064,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.615486,...,0.679715,-0.410313,-0.609153,-0.152425,1.405202,1.061462,-1.283888,0.493719,0.702770,1.698639
190,1.065069,1.702017,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,0.550554,...,1.435026,0.175592,-0.525307,-0.355465,1.399758,1.049897,-1.285265,0.471754,0.684734,1.730727


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(360, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180, return_sequences=True, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[4099535289, 3189208118, 3550699967, 1610092690, 2067431773, 1526704238, 3877375562, 3150458858, 3809890522, 872394471, 2495304199, 744760995, 548568874, 1790663462, 4059751872, 3034476847, 758541248, 2702557435, 4275931240, 2191870968, 2948392614, 2501359560, 4178907912, 3292638153, 3027599812, 674951027, 200341156, 3856566672, 3262014669, 1734498621, 603194504, 3851067747, 665451573, 2714063880, 2542421875, 2564596611, 3690362840, 4149187587, 3972144108, 978254291, 832027006, 1987004694, 2825029657, 1709966413, 3925345435, 2471736596, 2369629234, 3763078707, 3681445350, 2166264074]


Step: 0 ___________________________________________
val_loss: 154.2418975830078
winner_seed: 4099535289


Step: 1 ___________________________________________
val_loss: 194.9348907470703


Step: 2 ___________________________________________
val_loss: 108.67005920410156
winner_seed: 3550699967


Step: 3 ___________________________________________
val_loss: 187.54959106445312


Step: 4 _____________________

In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 159ms/step - loss: 25196.1895 - val_loss: 20580.5391
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 12773.9209 - val_loss: 3866.3818
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 2609.2896 - val_loss: 1964.8535
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 1833.2963 - val_loss: 1849.1049
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 1796.4351 - val_loss: 1762.7661
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 1760.1958 - val_loss: 1749.3342
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 1758.1222 - val_loss: 1678.3201
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 1750.2401 - val_loss: 1709.3138
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 1745.2047 - val_loss: 1678.6284
Epoch 10/10000
4/4 [======================

4/4 [==============================] - 0s 9ms/step - loss: 678.9785 - val_loss: 745.0411
Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 684.4604 - val_loss: 742.5411
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 641.6554 - val_loss: 726.6295
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 586.8332 - val_loss: 721.1450
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 688.2012 - val_loss: 879.2144
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 751.9946 - val_loss: 796.3552
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 674.9242 - val_loss: 740.8594
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 651.8332 - val_loss: 608.2731
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 590.9169 - val_loss: 584.3046
Epoch 88/10000
4/4 [==============================] - 0s 9ms/step - loss: 586.5

4/4 [==============================] - 0s 9ms/step - loss: 517.0953 - val_loss: 452.0486
Epoch 158/10000
4/4 [==============================] - 0s 9ms/step - loss: 513.6175 - val_loss: 456.6909
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 510.5130 - val_loss: 452.0207
Epoch 160/10000
4/4 [==============================] - 0s 9ms/step - loss: 513.3022 - val_loss: 451.7347
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 517.1637 - val_loss: 449.8875
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 507.9544 - val_loss: 500.0563
Epoch 163/10000
4/4 [==============================] - 0s 9ms/step - loss: 528.9521 - val_loss: 457.8981
Epoch 164/10000
4/4 [==============================] - 0s 9ms/step - loss: 512.4376 - val_loss: 453.8220
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 510.6659 - val_loss: 449.2091
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - l

4/4 [==============================] - 0s 9ms/step - loss: 515.2909 - val_loss: 449.4557
Epoch 236/10000
4/4 [==============================] - 0s 9ms/step - loss: 514.8097 - val_loss: 394.7345
Epoch 237/10000
4/4 [==============================] - 0s 9ms/step - loss: 496.1266 - val_loss: 484.5778
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 515.4721 - val_loss: 481.5399
Epoch 239/10000
4/4 [==============================] - 0s 9ms/step - loss: 515.9377 - val_loss: 487.5768
Epoch 240/10000
4/4 [==============================] - 0s 9ms/step - loss: 515.9874 - val_loss: 425.1211
Epoch 241/10000
4/4 [==============================] - 0s 9ms/step - loss: 514.3572 - val_loss: 427.6284
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 505.0803 - val_loss: 391.9718
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 506.6036 - val_loss: 405.5829
Epoch 244/10000
4/4 [==============================] - 0s 9ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 458.9339 - val_loss: 458.7172
Epoch 314/10000
4/4 [==============================] - 0s 9ms/step - loss: 501.9360 - val_loss: 591.9481
Epoch 315/10000
4/4 [==============================] - 0s 9ms/step - loss: 578.8478 - val_loss: 454.6030
Epoch 316/10000
4/4 [==============================] - 0s 9ms/step - loss: 488.0714 - val_loss: 577.1238
Epoch 317/10000
4/4 [==============================] - 0s 9ms/step - loss: 506.9840 - val_loss: 435.6579
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 457.4406 - val_loss: 463.2834
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 450.6789 - val_loss: 447.2672
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 450.1503 - val_loss: 436.1428
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 439.4572 - val_loss: 418.0509
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - lo

...

4/4 [==============================] - 0s 9ms/step - loss: 24.6577 - val_loss: 256.8440
Epoch 1173/10000
4/4 [==============================] - 0s 9ms/step - loss: 32.2671 - val_loss: 278.6248
Epoch 1174/10000
4/4 [==============================] - 0s 9ms/step - loss: 28.2835 - val_loss: 283.8303
Epoch 1175/10000
4/4 [==============================] - 0s 9ms/step - loss: 35.4978 - val_loss: 236.2501
Epoch 1176/10000
4/4 [==============================] - 0s 9ms/step - loss: 33.1265 - val_loss: 174.8962
Epoch 1177/10000
4/4 [==============================] - 0s 9ms/step - loss: 32.2597 - val_loss: 184.9453
Epoch 1178/10000
4/4 [==============================] - 0s 9ms/step - loss: 34.8484 - val_loss: 256.6908
Epoch 1179/10000
4/4 [==============================] - 0s 9ms/step - loss: 29.0805 - val_loss: 241.5732
Epoch 1180/10000
4/4 [==============================] - 0s 9ms/step - loss: 26.9114 - val_loss: 277.1177
Epoch 1181/10000
4/4 [==============================] - 0s 9ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 24.5172 - val_loss: 223.0765
Epoch 1251/10000
4/4 [==============================] - 0s 9ms/step - loss: 24.7413 - val_loss: 203.1392
Epoch 1252/10000
4/4 [==============================] - 0s 9ms/step - loss: 27.9951 - val_loss: 233.5590
Epoch 1253/10000
4/4 [==============================] - 0s 9ms/step - loss: 27.5049 - val_loss: 235.3282
Epoch 1254/10000
4/4 [==============================] - 0s 9ms/step - loss: 25.0797 - val_loss: 223.6656
Epoch 1255/10000
4/4 [==============================] - 0s 9ms/step - loss: 22.1458 - val_loss: 206.1118
Epoch 1256/10000
4/4 [==============================] - 0s 9ms/step - loss: 21.9974 - val_loss: 224.3285
Epoch 1257/10000
4/4 [==============================] - 0s 9ms/step - loss: 18.4770 - val_loss: 263.4298
Epoch 1258/10000
4/4 [==============================] - 0s 9ms/step - loss: 23.5496 - val_loss: 236.0471
Epoch 1259/10000
4/4 [==============================] - 0s 9ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 18.7057 - val_loss: 197.8821
Epoch 1329/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.2078 - val_loss: 208.9052
Epoch 1330/10000
4/4 [==============================] - 0s 9ms/step - loss: 21.7256 - val_loss: 248.0463
Epoch 1331/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.0557 - val_loss: 234.0729
Epoch 1332/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.2641 - val_loss: 228.8397
Epoch 1333/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.8816 - val_loss: 233.3273
Epoch 1334/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.1457 - val_loss: 223.0888
Epoch 1335/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.5014 - val_loss: 248.7222
Epoch 1336/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.0906 - val_loss: 263.9118
Epoch 1337/10000
4/4 [==============================] - 0s 9ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 12.9375 - val_loss: 198.5571
Epoch 1407/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.3439 - val_loss: 291.3609
Epoch 1408/10000
4/4 [==============================] - 0s 9ms/step - loss: 14.3705 - val_loss: 211.9318
Epoch 1409/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.2694 - val_loss: 176.2410
Epoch 1410/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.6311 - val_loss: 200.9618
Epoch 1411/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.1740 - val_loss: 234.8067
Epoch 1412/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.2687 - val_loss: 181.2907
Epoch 1413/10000
4/4 [==============================] - 0s 9ms/step - loss: 18.5915 - val_loss: 202.1293
Epoch 1414/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.4975 - val_loss: 191.8533
Epoch 1415/10000
4/4 [==============================] - 0s 9ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 10.5949 - val_loss: 201.6811
Epoch 1485/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.8561 - val_loss: 185.1983
Epoch 1486/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.9980 - val_loss: 181.6381
Epoch 1487/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.4912 - val_loss: 176.8389
Epoch 1488/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.9221 - val_loss: 175.9926
Epoch 1489/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.1082 - val_loss: 160.3997
Epoch 1490/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.5101 - val_loss: 150.1975
Epoch 1491/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.7218 - val_loss: 161.7587
Epoch 1492/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.0370 - val_loss: 176.6773
Epoch 1493/10000
4/4 [==============================] - 0s 9ms/step - loss: 

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,188.561295,191.261642,191.774399,191.367569,190.13707,185.565018,154.392166,137.696152,137.264313,155.007736,159.462387,161.519104,174.064728,183.293961,187.58992,189.069962,188.011063,181.882065,147.334885,137.589813,139.162109,157.075638,156.84285,156.958481,160.897263,165.822174,171.423019,168.210037,157.270157,140.381866
Target,179.243,194.134,186.948,194.907,179.505,173.278,160.261,113.987,114.289,117.313,110.502,147.996,152.386,177.259,155.913,172.41,179.419,157.165,151.601,129.537,105.236,110.835,127.925,127.138,157.128,164.44,140.298,168.407,163.615,155.07
Error,9.318298,2.87236,4.826401,3.539429,10.632065,12.287018,5.868835,23.709152,22.975311,37.694733,48.960388,13.523102,21.678726,6.034958,31.676926,16.659958,8.592056,24.717072,4.266113,8.052811,33.926109,46.240639,28.917847,29.82048,3.769257,1.382172,31.125015,0.19696,6.344849,14.688141


In [38]:
display(mae)
display(mape)

17.143238

0.13009937

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined